In [1]:
import numpy as np
import pandas as pd
import networkx as nx

from sklearn.cluster import KMeans
from sklearn import manifold

from scipy.optimize import minimize
from scipy.spatial.distance import pdist, squareform

import matplotlib.pyplot as plt
# matlab io
import scipy.io as sio

import multiprocessing as mp

%matplotlib inline

In [2]:
orderdf = pd.read_csv('ignored_assets/orders.csv', parse_dates = [6,7])
coredf = orderdf[orderdf['dest_district_hash'].isin(orderdf['start_district_hash'].unique())].copy()


In [3]:
T=24

In [4]:
# Pick one day: according to the ICRA paper, January 21, 2016 has the highest activity
coredfs=coredf[(coredf['timestamp']>='2016-01-21') & (coredf['timestamp']<'2016-01-22')].copy()

In [5]:
districts = coredfs['start_district_hash'].unique()
N = len(districts)


In [6]:
# pos_i is already in inferred_locations.csv.pos_i, a Nx2 vector of node locations
pos_i=np.zeros([N,2])
posdf=pd.read_csv('inferred_locations.csv')
posdf=posdf.set_index('start_district_hash')
for i in range(N):
    pos_i[i,0]=posdf['x'][districts[i]]
    pos_i[i,1]=posdf['y'][districts[i]]

In [7]:
# Append travel time to every trip
def district_loc(district_hash):
    return np.array([posdf.loc[district_hash].x, posdf.loc[district_hash].y])
def travel_distance(_record):
    return np.linalg.norm(district_loc(_record['start_district_hash'])- district_loc(_record['dest_district_hash']))
def travel_distance_pd(df):
    return df.apply(travel_distance,axis=1)

In [8]:
#my_travel_distances = coredfs.apply(travel_distance,axis=1); #distances are in km



In [9]:
def parallelize_dataframe(df, func,num_partitions=mp.cpu_count()):
    df_split = np.array_split(df, num_partitions)
    assert mp.cpu_count()>2, "With 2 or fewer cores, you probably don't want to run this!"
    pool = mp.Pool(mp.cpu_count()-2)
    df_out = pd.concat(pool.map(func, df_split))
    pool.close()
    pool.join()
    return df_out


In [10]:
my_travel_distances = parallelize_dataframe(coredfs,travel_distance_pd)

In [11]:
coredfs['travel_distance']=my_travel_distances

In [34]:
# Compute departure rates hour-by-hour. The output is lambda_tv, a NxNxT matrix of departure rates denoting the departure rate per minute
lambda_tv = np.zeros([N,N,T]) 
#For each departure location, compute probability of destinations. Output is pij_tv, a NxNXT matrix of o-d probabilities
pij_tv = np.zeros([N,N,T])
# speed: compute for each trip, then avg. # speed is in km/minute
speed = np.zeros([T,1])
#compute avg expected travel time. Output is Tij_tv, a NxNxT vector of travel times (in minutes).
Tij_tv = np.zeros([N,N,T])


for t in range(T):
    print ('2016-01-21 '+str(t)+':00')
    coredfs_h = coredfs[(coredfs['timestamp']>=('2016-01-21 '+str(t)+':00:00')) & (coredfs['timestamp']<('2016-01-21 '+str(t)+':59:59'))]
    tempcdfs=coredfs_h.groupby(['start_district_hash','dest_district_hash']).count()['order_id']
    tempttime = coredfs_h.groupby(['start_district_hash','dest_district_hash']).mean()['expected_travel_time']
    
    #temptottime = coredfs_h.groupby(['start_district_hash']).sum()['expected_travel_time']
    #temptotdist = coredfs_h.groupby(['start_district_hash']).sum()['travel_distance']
    for od in range(N):        
        for dd in range(N):
            try:
                lambda_tv[od,dd,t] = tempcdfs[districts[od],districts[dd]]/60.
            except:
                lambda_tv[od,dd,t] = 0
            try:
                Tij_tv[od,dd,t] = tempttime[districts[od],districts[dd]]
            except:
                Tij_tv[od,dd,t] = np.nan
        if sum(lambda_tv[od,:,t]):
            for dd in range(N):
                pij_tv[od,dd,t]=lambda_tv[od,dd,t]/sum(lambda_tv[od,:,t])
                
    speed[t]=sum(coredfs_h['travel_distance'])/sum(coredfs_h['expected_travel_time'])

2016-01-21 0:00
2016-01-21 1:00
2016-01-21 2:00
2016-01-21 3:00
2016-01-21 4:00
2016-01-21 5:00
2016-01-21 6:00
2016-01-21 7:00
2016-01-21 8:00
2016-01-21 9:00
2016-01-21 10:00
2016-01-21 11:00
2016-01-21 12:00
2016-01-21 13:00
2016-01-21 14:00
2016-01-21 15:00
2016-01-21 16:00
2016-01-21 17:00
2016-01-21 18:00
2016-01-21 19:00
2016-01-21 20:00
2016-01-21 21:00
2016-01-21 22:00
2016-01-21 23:00


In [35]:
for od in range(N):        
    for dd in range(N): 
        meanTT=np.average(Tij_tv[od,dd,np.logical_not(np.isnan(Tij_tv[od,dd,:]))])
        if np.isnan(meanTT): #Fuck it
            for t in range(T):
                meanTTt = np.linalg.norm(pos_i[od]-pos_i[dd],ord=2)/speed[t]
                Tij_tv[od,dd,t]=meanTTt
        else:
            Tij_tv[od,dd,np.isnan(Tij_tv[od,dd,:])]=meanTT

In [36]:
sio.savemat('timeVaryingSimDataHangzhou'+str(N)+'Stations.mat',{
        'lambda_tv':lambda_tv,
        'pij_tv': pij_tv,
        'pos_i': pos_i,
        'speed_tv': speed,
        'Tij_tv': Tij_tv
        })

In [33]:
Tij_tv

array([[[   4.57644518,    3.88815841,    3.79484261, ...,    3.39195164,
            3.42162461,    4.07302934],
        [  13.27858641,   13.73075371,    5.50563231, ...,    9.00058355,
            8.26458767,   10.30590695],
        [  55.52290214,   64.38790332,   55.52290214, ...,   55.52290214,
           55.52290214,   55.52290214],
        ..., 
        [  54.7372643 ,   52.31704862,   46.70633325, ...,   40.23012453,
           42.18425234,   55.93046075],
        [  48.49688915,   46.35259252,   41.38153222, ...,   35.64364999,
           37.37499555,   49.55405407],
        [  67.28465124,   64.30965111,   57.41279516, ...,   49.4520494 ,
           51.85412063,   68.75136331]],

       [[  13.07793518,   14.75863078,    8.47618534, ...,   10.14796313,
            9.8556484 ,   14.20865135],
        [   5.05229614,    4.56349202,    4.08051993, ...,    4.17524005,
            4.13467359,    5.23705858],
        [  29.53715913,   29.53715913,   29.53715913, ...,   28.21469925